In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from gurobipy import *
from matplotlib import pyplot as plt
import collections
from copy import *
from IPython.display import HTML
from IPython.display import display
from collections import *

In [2]:
# 'skus' table
skus = pd.read_csv('JD_sku_data.csv')
# 'orders' table
orders = pd.read_csv('JD_order_data.csv')
# 'delivery' table
delivery = pd.read_csv('JD_delivery_data.csv')
# 'inventory' table
inventory = pd.read_csv('JD_inventory_data.csv')
# 'network' table
network = pd.read_csv('JD_network_data.csv')

# Data Processing

We consider one region as instance

In [3]:
sample_dcs = sorted(network[network['region_ID'] == 3]['dc_ID'].tolist())
sample_dcs

[3, 14, 33, 34, 35, 64]

In [4]:
orders_in_sample_dcs = orders[(orders['dc_des'].isin(sample_dcs)) & (orders['order_date'] <= '2018-03-31')]

In [5]:
skus_in_sample_dcs = orders_in_sample_dcs.groupby('sku_ID').count()['order_ID'].to_dict()

We choose top SKUs 

In [6]:
demand_over_skus = orders_in_sample_dcs.groupby('sku_ID').count()['order_ID'].to_dict()
sample_skus = [i for i, j in sorted(demand_over_skus.items(), key = lambda kv: kv[1], reverse=True)]

sample_skus = sample_skus[:10]

In [11]:
sample_orders = orders_in_sample_dcs[orders_in_sample_dcs['sku_ID'].isin(sample_skus)]
demand_over_skus = sample_orders.groupby('sku_ID').count()['order_ID'].to_dict()
demand_over_dcs = sample_orders.groupby('dc_des').count()['order_ID'].to_dict()
sample_skus = [i for i, j in sorted(demand_over_skus.items(), key = lambda kv: kv[1], reverse=True)]

In [12]:
len(sample_skus)

10

In [13]:
demand_dist = {}
for i in sample_dcs:
    for k in sample_skus:
        hist = []
        for d in range(1, 32):
            if d <= 9:
                date = '2018-03-0' + str(d)
            else:
                date = '2018-03-' + str(d)
            hist.append(len(sample_orders[(sample_orders['dc_des'] == i) & (sample_orders['sku_ID'] == k) & (sample_orders['order_date'] == date)]))
            
        demand_dist[i, k] = (np.mean(hist), np.std(hist))

We generate demand scenarios

In [14]:
total_demand = sum(demand_over_skus.values()) 
T = int((np.ceil(total_demand / 1000)) * 1000) 
sku_freq = []
for k in sample_skus:
    if k in demand_over_skus:
        sku_freq.append(demand_over_skus[k] / T)
    else:
        sku_freq.append(0)
dc_freq = [demand_over_dcs[dc] / total_demand for dc in sample_dcs]
multinomial_sku = [1 - sum(sku_freq)] + sku_freq
multinomial_dc = dc_freq

In [15]:
N = 1000
demand_scenarios = []
demand_scenarios_agg = []

for n in range(N):
    skus = np.random.choice(['NA'] + sample_skus, T, p=multinomial_sku)
    dcs = np.random.choice(sample_dcs, T, p=multinomial_dc)
    sample = list(zip(skus, dcs))
        
    demand_scenarios.append(sample)

#     D = {}
#     for i in sample_dcs:
#         for k in sample_skus:
#             D[k, i] = max(int(sum(np.random.normal(demand_dist[i, k][0], demand_dist[i, k][1], 7))), 0)
#     demand_scenarios_agg.append(D)

expected = {}
for k in range(len(sku_freq)):
    for i in range(len(dc_freq)):
        expected[sample_skus[k], sample_dcs[i]] = T * sku_freq[k] * dc_freq[i]
    

We can also consider a special case 

In [457]:
# sample_dcs = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H']
# sample_skus = [i for i in range(1, 9)]

sample_dcs = ['A', 'B', 'C', 'D']
sample_skus = [i for i in range(1, 9)]

In [459]:
N = 1000
    
demand_scenarios = []
sku_mean = np.arange(100, 120, 0.1)[::-1]
sku_std = np.arange(0, 50, 5)[::-1]
dc_freq = [1/len(sample_dcs)] * len(sample_dcs)

for n in range(N):
    skus = []
    for k in sample_skus:
        skus += [k] * int(max(0, np.random.normal(sku_mean[sample_skus.index(k)], sku_std[sample_skus.index(k)])))
    skus = np.random.permutation(skus)    
        
    dcs = np.random.choice(sample_dcs, len(skus), p=dc_freq)
    sample = list(zip(skus, dcs))
    demand_scenarios.append(sample)  
    
expected = {}
for k in range(len(sample_skus)):
    for i in range(len(sample_dcs)):
        expected[sample_skus[k], sample_dcs[i]] = sku_mean[k] * dc_freq[i]

## Model Formulation

In [23]:
def optimize(S, D): 
    model = Model('model')
    model.setParam("LogToConsole", 0)

    fulfill = model.addVars(((k, i, j) for k in sample_skus for i in supply for j in demand), vtype=GRB.CONTINUOUS, name='fulfill')

    model.setObjective(sum(fulfill[k, i, j] * cost[k, i, j] for k, i, j in fulfill), GRB.MINIMIZE)

    for k in sample_skus:
        for i in supply:
            model.addConstr(fulfill.sum(k, i, '*') <= S[k, i], name='supply_%s_%s' % (k, i))

        for j in demand:
            model.addConstr(fulfill.sum(k, '*', j) == D[k, j], name='demand_%s_%s' % (k, j))

    model.optimize()
    
    return model, fulfill

def optimize_k(k, S, D): 
    model = Model('model_1')
    model.setParam("LogToConsole", 0)

    fulfill = model.addVars([(i, j) for i in supply for j in demand], vtype=GRB.CONTINUOUS, name='fulfill_1')

    model.setObjective(sum(fulfill[i, j] * cost[k, i, j] for i, j in fulfill), GRB.MINIMIZE)

    for i in supply:
        model.addConstr(sum(fulfill[i, j] for j in demand) <= S[k, i])

    for j in demand:
        model.addConstr(sum(fulfill[i, j] for i in supply) == D[k, j])

    model.optimize()
    
    return model, fulfill

def misp_k(k, D, _lambda): 
    model = Model()
    model.setParam("LogToConsole", 0)

    Z = model.addVar(vtype=GRB.BINARY, name='indicator')
    S = model.addVars([i for i in supply], vtype=GRB.CONTINUOUS, name='inventory')
    F = model.addVars([(m, i, j) for m in range(len(D)) for i in supply for j in demand], vtype=GRB.CONTINUOUS, name='fulfill')
    
    allocation_cost = sum(S[i] * b for i in supply[1:])
    fulfill_cost = sum(F[m, i, j] * cost[k, i, j] for m in range(len(D)) for i in supply for j in demand) / len(D)
    penalty_cost = _lambda * Z
    
    model.setObjective(allocation_cost + fulfill_cost + penalty_cost, GRB.MINIMIZE)

    model.addConstr(S[0] == 10000)
    
    for m in range(len(D)):
        for i in supply:
            model.addConstr(F.sum(m, i, '*') <= S[i])

        for j in demand:
            model.addConstr(F.sum(m, '*', j) == D[m][j])
                 
    model.addConstr(sum(S[i] for i in supply[1:]) <= 10000 * Z)
        
    model.optimize()
    
    S_k = model.getAttr('X', S)
    #Z_k = model.getAttr('X', Z)
    
    return S_k

We set up parameters

In [24]:
b = 1

supply = [0] + sample_dcs
demand = sample_dcs
cost = {}
for k in sample_skus:
    for j in demand:
        for i in supply:              
            if i == 0:
                cost[k, i, j] = np.random.normal(10, 0)
            else:
                if i == j:
                    cost[k, i, j] = np.random.normal(2, 0)
                elif i == sample_dcs[0]:
                    cost[k, i, j] = np.random.normal(4, 0)
#                 elif (i, j) in [(33, 34), (34, 35), (35, 64), (64, 33)]:    
#                     cost[k, i, j] = np.random.normal(4, 0)
                else:
                    cost[k, i, j] = np.random.normal(6, 0)
            


We optimize inventory allocation with capacity constraints

In [35]:
unit = {}
for i in supply[1:]:
    if i == sample_dcs[0]:
        unit[i] = 100000
    else:
        unit[i] = 100000
        
S_init = defaultdict(int)

for k in sample_skus:
    S_init[k, 0] = 1e8 
    for i in supply[1:]:
        #S_init[k, i] = int(expected[k, i])
        S_init[k, i] = 0

In [38]:
solve_allocation_coordinate()

3 [117, 88, 67, 0, 0, 0, 0, 0, 0, 0]
14 [25, 19, 14, 0, 0, 0, 0, 0, 0, 0]
33 [393, 295, 212, 0, 0, 0, 0, 0, 0, 0]
34 [207, 154, 112, 0, 0, 0, 0, 0, 0, 0]
35 [204, 149, 109, 0, 0, 0, 0, 0, 0, 0]
64 [212, 156, 116, 0, 0, 0, 0, 0, 0, 0]
------------------
3 [117, 88, 67, 0, 0, 0, 0, 0, 0, 0]
14 [25, 19, 14, 0, 0, 0, 0, 0, 0, 0]
33 [393, 295, 212, 0, 0, 0, 0, 0, 0, 0]
34 [207, 154, 112, 0, 0, 0, 0, 0, 0, 0]
35 [204, 149, 109, 0, 0, 0, 0, 0, 0, 0]
64 [212, 156, 116, 0, 0, 0, 0, 0, 0, 0]
------------------
3 [117, 88, 67, 0, 0, 0, 0, 0, 0, 0]
14 [25, 19, 14, 0, 0, 0, 0, 0, 0, 0]
33 [393, 295, 212, 0, 0, 0, 0, 0, 0, 0]
34 [207, 154, 112, 0, 0, 0, 0, 0, 0, 0]
35 [204, 149, 109, 0, 0, 0, 0, 0, 0, 0]
64 [212, 156, 116, 0, 0, 0, 0, 0, 0, 0]
------------------


In [37]:
def solve_allocation_coordinate():
    for tau in range(3):
        for k in sample_skus:
            D = []
            for _iter in range(200):
                sample_path = demand_scenarios[_iter]
                D_ins = collections.defaultdict(int)
                for pair in sample_path:
                    if k == pair[0]:
                        D_ins[pair[1]] += 1
                D.append(D_ins)  

#             critical = {}
#             for i in sample_dcs:
#                 position = [S_init[l, i] for l in sample_skus if l != k]
#                 if card[i] < len(sample_skus):
#                     critical[i] = sorted(position, reverse=True)[card[i]-1]
#                 else:
#                     crtitcal[i] = 0
            
            _lambda = 4000
            S_k = misp_k(k, D, _lambda)
                
            for i in sample_dcs:
                S_init[k, i] = S_k[i]
                                 
        for i in supply[1:]:
            res = []
            for k in sample_skus:
                res.append(int(S_init[k, i]))
            print(i, res)
        print('------------------')

In [653]:
allocation = []
for i in supply[1:]:
    res = []
    for k in sample_skus:
        if int(S_init[k, i]) <= 0:
            Q = 0
            S_init[k, i] = 0
        else:
            Q = int(S_init[k, i])
        res.append(Q)
    allocation.append(res)
allocation = np.array(allocation)

In [654]:
allocation

array([[288, 213,   0, 208, 202, 202, 197, 195,  45,  46,  46,  45,   0,
         46,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,  41,   0,  41,  29,  42],
       [242, 175,  68,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,  13,   0],
       [222, 165,  50,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,  42,   0,   0,   0,   0,   0],
       [206, 153,  47,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,  41,   0,   0,   0],
       [188, 139,  43,   0,   0,   0,   0,   0,   0,   0,   0,   0,  43,
          0,   0,   0,   0,   0,   0,   0]])

We present the results

In [1361]:
df = pd.read_excel('300sku_quantity_v3.xlsx')
allocation = np.array(df)
for k in sample_skus:
    for i in sample_dcs:
        idx_k = sample_skus.index(k)
        idx_i = sample_dcs.index(i)
        S_init[k, i] = allocation[idx_i, idx_k]

In [658]:
import pandas as pd
df = pd.DataFrame(allocation)
df.to_excel('20sku_quantity_v2.xlsx', index=False)

We simulate online fulfillment

In [561]:
offline_value, online_value, online_value_greedy = [], [], []
offline_fulfill, online_fulfill = defaultdict(int), defaultdict(int)

np.random.seed(1234)
for _iter in range(10):    
    print('---iter ' + str(_iter) + '---')
    sample_path = demand_scenarios[_iter]
    
    #offline
    D = collections.defaultdict(int)
    for pair in sample_path:
        k, i = pair[0], pair[1]
        D[k, i] += 1

    #D = demand_scenarios_agg[_iter]
    model, fulfill = optimize(S_init, D)
    offline_value.append(model.getAttr('objVal'))
        
    print(offline_value[_iter])
    
    # online 
    obj_bayes = 0
    obj_greedy = 0
    S_bayes = deepcopy(S_init)
    S_greedy = deepcopy(S_init)
#     for t in range(T): 
#         # serve demand
#         d = sample_path[t]
#         if d[0] != 'NA':
#             ### greedy 
#             i = sorted([(i, cost[d[0], i, d[1]]) for i in supply if S_greedy[d[0], i] >= 1], key=lambda x:x[1])[0][0]
#             obj_greedy += cost[d[0], i, d[1]]
#             S_greedy[d[0], i] -= 1            
            
            
#             ### bayes selector          
#             if t == 0:
#                 D = collections.defaultdict(int)
#                 for k in sample_skus:
#                     for j in sample_dcs:
#                         D[k, j] = (T-t) * sku_freq_first[sample_skus.index(k)] * dc_freq_first[sample_dcs.index(j)]
                
#                 model, fulfill = optimize(S_bayes, D)
#                 _fulfill = model.getAttr('X', fulfill)

#                 policy = {}
#                 for k in sample_skus:
#                     for j in demand:
#                         flow = sorted([(i, _fulfill[k, i, j]) for i in supply], key=lambda x:x[1], reverse=True)
#                         policy[k, j] = flow[0][0]
            
#             elif t % 1 == 0:
#                 D = collections.defaultdict(int)
#                 for k in sample_skus:
#                     for j in sample_dcs:
#                         D[k, j] = (T-t) * sku_freq_first[sample_skus.index(k)] * dc_freq_first[sample_dcs.index(j)]                      
                
#                 model, fulfill = optimize_k(d[0], S_bayes, D)
#                 _fulfill = model.getAttr('X', fulfill)
#                 policy[d] = sorted([(i, _fulfill[i, d[1]]) for i in supply], key=lambda x:x[1], reverse=True)[0][0]         
                        
#             i = policy[d]
                     
#             if S_bayes[d[0], i] >= 1:
#                 obj_bayes += cost[d[0], i, d[1]]
#                 S_bayes[d[0], i] -= 1
#                 online_fulfill[(d[0], i, d[1])] += 1
#             else: 
#                 obj_bayes += cost[d[0], 0, d[1]]
#                 S_bayes[d[0], 0] -= 1
            
#     print(obj_bayes)
#     print(obj_greedy)
#     online_value.append(obj_bayes)
#     online_value_greedy.append(obj_greedy)
    
#     for i in supply[1:]:
#         res = []
#         for k in sample_skus:
#             res.append(int(S_bayes[k, i]))
#         print(i, res)
#     print('------------------')
    

---iter 0---
12206.0
---iter 1---
12572.0
---iter 2---
12368.0
---iter 3---
12572.0
---iter 4---
12534.0
---iter 5---
12356.0
---iter 6---
12336.0
---iter 7---
12430.0
---iter 8---
12390.0
---iter 9---
12154.0


In [562]:
np.mean(offline_value), np.mean(online_value), np.mean(online_value_greedy)

/home/nitu/anaconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/nitu/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


(12391.8, nan, nan)

In [563]:
sum(int(S_init[k, i]) for k in sample_skus for i in sample_dcs) * b + np.mean(offline_value)

15645.8

In [398]:
flexible = {1:[('G', 'A'), ('B', 'F')], 2:[('A', 'G'), ('D', 'B')], 3:[], 4:[], 5:[], 6:[]}
# flexible = {1:[('B', 'A'), ('C', 'B'), ('D', 'C'), ('E', 'D'), ('F', 'E'), ('G', 'F'), ('H', 'G'), ('A', 'H')], 2:[('B', 'A'), ('C', 'B'), ('D', 'C'), ('E', 'D'), ('F', 'E'), ('G', 'F'), ('H', 'G'), ('A', 'H')], 3:[], 4:[], 5:[], 6:[]}

In [420]:
sparse, remote = 0, 0
for k in sample_skus:
    if k == 3:
        for i in sample_dcs:
            for j in sample_dcs:
                if (i, j) in flexible[k] or cost[k, i, j] <= 4:
                    sparse += online_fulfill[k, i, j]
                else:
                    remote += online_fulfill[k, i, j]

In [421]:
sparse / (sparse + remote)

0.9962852897473997

## Appendix

In [566]:
# MIP formulation
def optimize_deterministic(D):  
    model = Model('mip')
    model.setParam("LogToConsole", 1)
    Z = model.addVars([(k, i) for k in sample_skus for i in supply[1:]], vtype=GRB.BINARY, name='indicator')
    S = model.addVars([(k, i) for k in sample_skus for i in supply], vtype=GRB.CONTINUOUS, name='inventory')
    F = model.addVars([(k, i, j) for k in sample_skus for i in supply for j in demand], vtype=GRB.CONTINUOUS, name='fulfill')
    
    allocation_cost = sum(S[k, i] * b for k in sample_skus for i in supply[1:])
    fulfill_cost = sum(F[k, i, j] * cost[k, i, j] for k in sample_skus for i in supply for j in demand)
    
    model.setObjective(allocation_cost + fulfill_cost, GRB.MINIMIZE)

    for k in sample_skus:
        model.addConstr(S[k, 0] == 10000)
        for i in supply:
            model.addConstr(F.sum(k, i, '*') <= S[k, i], name='supply_%s_%s' % (k, i))

        for j in demand:
            model.addConstr(F.sum(k, '*', j) == D[k, j], name='demand_%s_%s' % (k, j))
            
    for i in supply[1:]:
        #model.addConstr(Z.sum('*', i) <= card[i])
        for k in sample_skus:
            model.addConstr(S[k, i] <= 1000 * Z[k, i])
            
    for i in supply[1:]:
        model.addConstr(S.sum('*', i) <= unit[i])

    model.optimize()
    
    return model, Z, S, F

In [567]:
sample_mean = defaultdict(int)
for _iter in range(200):
    sample_path = demand_scenarios[_iter] 
    for pair in sample_path:
        k, i = pair[0], pair[1]
        sample_mean[k, i] += 1
#     D = demand_scenarios_agg[_iter]
#     for k in sample_skus:
#         for j in demand:
#             sample_mean[k, j] += D[k, j]

print('---')
D = {}
for k in sample_skus:
    for j in demand:
        D[k, j] = expected[k, j]
        
model, Z, S, F = optimize_deterministic(D)
_S = model.getAttr('X', S)
for i in supply[1:]:
    res = []
    for k in sample_skus:
        res.append(int(_S[k, i]))
    print(res)
    
S_init = _S

---
Parameter LogToConsole unchanged
   Value: 1  Min: 0  Max: 1  Default: 1
Optimize a model with 406 rows, 1100 columns and 2200 nonzeros
Variable types: 980 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [1e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e-01, 1e+05]
Found heuristic solution: objective 35280.000000
Presolve removed 394 rows and 1064 columns
Presolve time: 0.00s
Presolved: 12 rows, 36 columns, 72 nonzeros
Found heuristic solution: objective 18333.000000
Variable types: 36 continuous, 0 integer (0 binary)

Root relaxation: objective 1.058400e+04, 14 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    10584.000000 10584.0000  0.00%     -    0s

Explored 0 nodes (14 simplex iterations) in 0.02 seconds
Thread count was 4 (of 4 avail